In [ ]:
%load_ext autoreload
%autoreload 1
%aimport utils.utils
    
import argparse
import os
import json

from tqdm import tqdm

In [ ]:
import re
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import BaseOutputParser

In [ ]:
class DeepseekParser(BaseOutputParser):
    def parse(self, text):
        match =  re.search(r"```sql(.*?)```", text, flags=re.DOTALL)
        if match:
            return re.sub(r"\s+", ' ', match.group(1), flags=re.DOTALL).strip()
        else:
            raise ValueError("无法解析输出，请保证输出中包含 ```sql和```")

In [ ]:
QUESTION_FILE = "questions.json"
class args:
    model = "phi4"
    question = "/home/koushurui/Documents/Code/text2sql/DAIL-SQL/dataset/process/BIRD/TEST/SQL_9-SHOT_EUCDISQUESTIONMASK_QA-EXAMPLE_CTX-200_ANS-4096"
    start_index = 0
    end_index = 4
    batch_size =  1

In [ ]:
os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"
os.environ["OPENAI_API_KEY"] = "ollama"

In [ ]:
output_parser = DeepseekParser()
llm = ChatOpenAI(model="phi4", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     你是一个SQL语言大师，请你根据下面的输入，生成正确、可运行的SQL语句
     用
     ```sql
     code
     ```
     包裹生成的语句
     """), 
    ("human", "{input}")])

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
questions_json = json.load(open(os.path.join(args.question, QUESTION_FILE), "r"))
questions = [_["prompt"] for _ in questions_json["questions"]]
db_ids = [_["db_id"] for _ in questions_json["questions"]]

out_file = f"{args.question}/RESULTS_MODEL-{args.model}.txt"

In [ ]:
# output = chain.invoke(questions_json["questions"][0]["prompt"])
# with open(out_file, "w") as f:
#     for question in questions_json["questions"][1]:
#         f.write(chain.invoke(question["prompt"])+"\n")

In [ ]:
re.sub(r"\s+", ' ', "SELECT `Free Meal Count (Ages 5-17)` / `Enrollment (Ages 5-17)` AS Eligible_Free_Rate, School FROM frpm JOIN \n    schools USING (CDSCode)\nWHERE \n    `School Type` = 'Continuation'\nORDER BY \n    Eligible_Free_Rate ASC\nLIMIT 3;", flags=re.DOTALL)

In [24]:
print("/* Given the following database schema: */\nCREATE TABLE customers\n(\n    CustomerID INTEGER UNIQUE     not null\n        primary key,\n    Segment    TEXT null,\n    Currency   TEXT null\n)\nCREATE TABLE gasstations\n(\n    GasStationID INTEGER    UNIQUE   not null\n        primary key,\n    ChainID      INTEGER          null,\n    Country      TEXT null,\n    Segment      TEXT null\n)\nCREATE TABLE products\n(\n    ProductID   INTEGER   UNIQUE      not null\n        primary key,\n    Description TEXT null\n)\nCREATE TABLE \"transactions_1k\"\n(\n    TransactionID INTEGER\n        primary key autoincrement,\n    Date          DATE,\n    Time          TEXT,\n    CustomerID    INTEGER,\n    CardID        INTEGER,\n    GasStationID  INTEGER,\n    ProductID     INTEGER,\n    Amount        INTEGER,\n    Price         REAL\n)\nCREATE TABLE sqlite_sequence(name,seq)\nCREATE TABLE \"yearmonth\"\n(\n    CustomerID  INTEGER not null\n        references customers\n            on update cascade on delete cascade\n        references customers,\n    Date        TEXT    not null,\n    Consumption REAL,\n    primary key (Date, CustomerID)\n)\n\n/* Answer the following:\nWhat is the ratio of customers who pay in EUR against customers who pay in CZK?\nratio of customers who pay in EUR against customers who pay in CZK = count(Currency = 'EUR') / count(Currency = 'CZK').\\*\nSELECT ")

/* Given the following database schema: */
CREATE TABLE customers
(
    CustomerID INTEGER UNIQUE     not null
        primary key,
    Segment    TEXT null,
    Currency   TEXT null
)
CREATE TABLE gasstations
(
    GasStationID INTEGER    UNIQUE   not null
        primary key,
    ChainID      INTEGER          null,
    Country      TEXT null,
    Segment      TEXT null
)
CREATE TABLE products
(
    ProductID   INTEGER   UNIQUE      not null
        primary key,
    Description TEXT null
)
CREATE TABLE "transactions_1k"
(
    TransactionID INTEGER
        primary key autoincrement,
    Date          DATE,
    Time          TEXT,
    CustomerID    INTEGER,
    CardID        INTEGER,
    GasStationID  INTEGER,
    ProductID     INTEGER,
    Amount        INTEGER,
    Price         REAL
)
CREATE TABLE sqlite_sequence(name,seq)
CREATE TABLE "yearmonth"
(
    CustomerID  INTEGER not null
        references customers
            on update cascade on delete cascade
        references custome